In [ ]:
                                               #PART 1: Display 100 Top Words in Subset of 20 News Groups Data set

#Step 1 Import all the functions needed
from operator import index
import pandas as pd
from sklearn import datasets
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

#Step 2: Import the dataset witht the 20 NewsGroups
from sklearn.datasets import fetch_20newsgroups


newsgroups_train = fetch_20newsgroups()
print(newsgroups_train.keys())


FEATURES = newsgroups_train['filenames']
TARGET = 'titles'

FEATURES, TARGET

#Step 3: Take the Imported Newsgroups and transform the data into a dataframe.
print(newsgroups_train.keys())
FEATURES = newsgroups_train['filenames']
TARGET = 'titles'
FEATURES, TARGET
print(len(newsgroups_train['data']))
df = pd.DataFrame(newsgroups_train['data'])
df
df[TARGET] = newsgroups_train['target']
df

#Step 4: Perform Exploratory Data Analyis on the data
df.head()
df.isnull().sum()

#Step 5: Isolate the three needed subsets of the 20 NewsGroups, list them, & target the NewsGroups.
target_name_index = [newsgroups_train.target_names.index('talk.religion.misc'),newsgroups_train.target_names.index('sci.electronics'), newsgroups_train.target_names.index('rec.motorcycles') ]
word_index = np.where(newsgroups_train.target == target_name_index[0])[0]
speak_index = np.where(newsgroups_train.target == target_name_index[1])[0]
refrence_index = np.where(newsgroups_train.target == target_name_index [2])[0]
target_index = np.append(np.append(word_index, speak_index), refrence_index)
target_documents = [newsgroups_train.data[x] for x in target_index]

#Step 6: Display the top 100 Words
cv = CountVectorizer(stop_words='english', max_features=100)
words_bag = cv.fit_transform(target_documents)
sum_of_words = words_bag.sum(axis=0)
word_frequency = [(word, sum_of_words[0, idx]) for word, idx in cv.vocabulary_.items()]
word_frequency = sorted(word_frequency, key = lambda x: x[1], reverse=True)
print('Top 100 words across documents in electronics, religion, and motorcycles')
print('------------------------------------------------------------------------')
for word, count in word_frequency:
  print(word + ':', count)






In [ ]:
from inspect import EndOfBlock
                                                #PART 2: Cluster Documents (Unsupervised Learning) And Discover Topics
#Step 1 Import all the functions needed
from operator import index
import pandas as pd
from sklearn import datasets
from sklearn.datasets import fetch_20newsgroups
import numpy as np
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import NMF
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
from sklearn.cluster import KMeans
import nltk
#Step 2: Test if a word would count as a token
nltk.download('wordnet')
nltk.download('omw-1.4')


def letters_only(astr):
  return astr.isalpha()



#Step 3: Strip these words out of the corpus for the given topics and apply lemmatization. 
stripped_words = ['organization', 'article', 'mr', 'know', 'like', 'com', 'edu', 'lines', 'subject', 'university', 'say', 'think']
clean = []
lemmatizer = WordNetLemmatizer()
print('Cleaning the list')
for post in target_documents:
  clean.append(" ". join(lemmatizer.lemmatize(word.lower()) for word in post.split() if letters_only(word) and word.lower() not in stripped_words))
cleaned_words_bag = cv.fit_transform(clean)
print(cv.get_feature_names)


#Step #4: Find the optimal K
print('Finding the optimal K')
Sum_of_squared_distance = []
K = range(1,16)
for k in K:
  km = KMeans(n_clusters = k)
  km = km.fit(cleaned_words_bag)  
  km = Sum_of_squared_distance.append(km.inertia_)

#Step 5: Plotting the K Cluster
plt.plot(K, Sum_of_squared_distance)
plt.xlabel('k')
plt.ylabel(Sum_of_squared_distance)
plt.title('Elbow Method for the optimal K')
plt.show()



In [ ]:
#Step 6: Perform Topic Modeling (Must be ran seperatly from the K-Means Cluster)
K = 7
km = KMeans(n_clusters = K)
km = km.fit(cleaned_words_bag)
print('iter',km.n_iter_) #<----- Prining the number of iterations
print('features',km.n_features_in_) #<-------- Prining the number of features seen duirng the fit


for t in range (K-1):
   group_indices = np.where(km.labels_ == t )  
   group_docs = [clean[x] for x in group_indices[0]]
   if len(group_indices[0]) > 2:
     fits = cv.fit_transform(group_docs)

     print('Group' + str((t + 1))+':')
     nmf = NMF(n_components=3, random_state=50).fit(fits)
     for topic_idx, topic in enumerate(nmf.components_):
       print(topic_idx, ':', ' '.join([cv.get_feature_names()[x] for x in topic.argsort()[:-9:-1]]))
       continue
